In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf
from parselmouth.praat import call
import parselmouth
import librosa
from librosa import feature

In [2]:
# path to the directory of audio files

#Pataka_dir = r"Dataset/healthy_data/test_5_pataka_healthy"
Pataka_dir = r"Concussion/Final_Dataset_Without_Silence/Concussed_Dataset without Silence/Concussed_PaTaKa"
#Pataka_dir = r"test_5_pataka_healthy"

Pataka_audio = glob.glob(Pataka_dir + '/*.wav')

In [3]:
# Code for measuring pitch

def measurePitch(voice, f0min, f0max, unit):

    sound = parselmouth.Sound(voice) # read the sound
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
    return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [4]:
#measurePitch(Pataka_audio[0],75,300,'Hertz')
import statistics
from parselmouth.praat import call

In [5]:
def measureFormants(sound,f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median

In [6]:
#Spectral Feature Extraction

#stft used short-term Fourier transformation to compute Chroma features. STFT represents information about the classification of pitch and signal structure.
featureList1 = [
    feature.chroma_stft,
    feature.spectral_centroid,
    feature.spectral_bandwidth,
    feature.spectral_rolloff,
    feature.chroma_cqt,
    feature.chroma_cens,
    feature.melspectrogram,
    feature.mfcc,
    feature.spectral_contrast,
    feature.poly_features,
    feature.tonnetz,
]

featureList2 = [feature.zero_crossing_rate,
               feature.rms,
               feature.spectral_flatness]

def getFeatureVector(y,sr, order):  
    FLRes1 = [ np.mean(funct(y,sr)) for funct in featureList1]
    FLRes2 = [ np.mean(funct(y)) for funct in featureList2]

    featureVector =   FLRes1 + FLRes2
    return featureVector


def load_file_from_list(audio):
    y1, sr1 = librosa.load(audio, sr = None)
    return getFeatureVector(y = y1, sr = sr1, order = 16)

In [7]:
print(measureFormants(Pataka_audio[9],75,300))
print(measurePitch(Pataka_audio[9], 75, 300, 'Hertz'))
lst = load_file_from_list(Pataka_audio[9])
lst

(639.6516488026502, 1252.6951487175188, 2318.655704211161, 3237.6729867263202, 664.2192300738512, 1183.0215747349985, 2314.8068766291294, 3256.287619760415)
(4.4468125, 84.62635239126497, 3.8911593554606214, 8.426532965728333, 0.01833870613229295, 0.00021649974126701358, 0.007629419722967106, 0.00927034488760135, 0.022888259168901318, 0.10360515346450352, 0.9554832238390762, 0.030847527710242434, 0.05112265028598257, 0.2045214027274718, 0.0925425831307273)


[0.45033005,
 1418.8846473355964,
 1243.1201393442643,
 2515.456384892086,
 0.5916079,
 0.28298867,
 0.2532607,
 -26.762688,
 20.034210841753623,
 0.3552900161503918,
 0.0029788170347740243,
 0.1603564804406475,
 0.027023422,
 0.019698747]

In [8]:
#features_Pataka
lst[0]

0.5589247

In [8]:
def extract_feature(Audio):
    file_list = []
    duration_list = []
    mean_F0_list = []
    sd_F0_list = []
    hnr_list = []
    localJitter_list = []
    localabsoluteJitter_list = []
    rapJitter_list = []
    ppq5Jitter_list = []
    ddpJitter_list = []
    localShimmer_list = []
    localdbShimmer_list = []
    apq3Shimmer_list = []
    aqpq5Shimmer_list = []
    apq11Shimmer_list = []
    ddaShimmer_list = []
    f1_mean_list = []
    f2_mean_list = []
    f3_mean_list = []
    f4_mean_list = []
    f1_median_list = []
    f2_median_list = []
    f3_median_list = []
    f4_median_list = []
    #Spectrul_Features
    chroma_stft = []
    spectral_centroid= []
    spectral_bandwidth= []
    spectral_rolloff= []
    chroma_cqt= []
    chroma_cens= []
    melspectrogram= []
    mfcc= []
    spectral_contrast= []
    poly_features= []
    tonnetz= []
    zero_crossing_rate= []
    rms= []
    spectral_flatness= []
    

    
    cnt=1
    for i in Audio:
        aud_id = i.split('.')[0].strip()
        aud_id = aud_id.split('\\')[1]
        print(cnt,aud_id)
        cnt = cnt+1
        
        #Pitch Features
        (duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, 
        localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(i, 75, 300, 'Hertz')
        #sound = parselmouth.Sound(i)
        
        #Formants Features
        (f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median) = measureFormants(i, 75, 300)
        file_list.append(aud_id) # make an ID list
        
        
        #Spectral Features Extraction
        spectral_feature_list = load_file_from_list(i)
        chroma_stft.append(spectral_feature_list[0])
        spectral_centroid.append(spectral_feature_list[1])
        spectral_bandwidth.append(spectral_feature_list[2])
        spectral_rolloff.append(spectral_feature_list[3])
        chroma_cqt.append(spectral_feature_list[4])
        chroma_cens.append(spectral_feature_list[5])
        melspectrogram.append(spectral_feature_list[6])
        mfcc.append(spectral_feature_list[7])
        spectral_contrast.append(spectral_feature_list[8])
        poly_features.append(spectral_feature_list[9])
        tonnetz.append(spectral_feature_list[10])
        zero_crossing_rate.append(spectral_feature_list[11])
        rms.append(spectral_feature_list[12])
        spectral_flatness.append(spectral_feature_list[13])
        
        
        duration_list.append(duration) # make duration list
        mean_F0_list.append(meanF0) # make a mean F0 list
        sd_F0_list.append(stdevF0) # make a sd F0 list
        hnr_list.append(hnr) #add HNR data
        
        # add raw jitter and shimmer measures
        localJitter_list.append(localJitter)
        localabsoluteJitter_list.append(localabsoluteJitter)
        rapJitter_list.append(rapJitter)
        ppq5Jitter_list.append(ppq5Jitter)
        ddpJitter_list.append(ddpJitter)
        localShimmer_list.append(localShimmer)
        localdbShimmer_list.append(localdbShimmer)
        apq3Shimmer_list.append(apq3Shimmer)
        aqpq5Shimmer_list.append(aqpq5Shimmer)
        apq11Shimmer_list.append(apq11Shimmer)
        ddaShimmer_list.append(ddaShimmer)
        
        # add the formant data
        f1_mean_list.append(f1_mean)
        f2_mean_list.append(f2_mean)
        f3_mean_list.append(f3_mean)
        f4_mean_list.append(f4_mean)
        f1_median_list.append(f1_median)
        f2_median_list.append(f2_median)
        f3_median_list.append(f3_median)
        f4_median_list.append(f4_median)
    
    #Load features in data frame
                       #pd.DataFrame(np.column_stack([duration_list1,
    features_Pataka = pd.DataFrame(np.column_stack([file_list, duration_list, mean_F0_list, sd_F0_list, hnr_list, 
                                   localJitter_list, localabsoluteJitter_list, rapJitter_list, 
                                   ppq5Jitter_list, ddpJitter_list, localShimmer_list, 
                                   localdbShimmer_list, apq3Shimmer_list, aqpq5Shimmer_list, 
                                   apq11Shimmer_list, ddaShimmer_list, f1_mean_list, 
                                   f2_mean_list, f3_mean_list, f4_mean_list, 
                                   f1_median_list, f2_median_list, f3_median_list, 
                                   f4_median_list, chroma_stft, spectral_centroid, spectral_bandwidth, spectral_rolloff,
                                   zero_crossing_rate, chroma_cqt, chroma_cens, melspectrogram, mfcc, spectral_contrast, 
                                   poly_features, tonnetz, rms, spectral_flatness]),
                                   columns=['ID_audio','duration', 'meanF0Hz', 'stdevF0Hz', 'HNR', 
                                            'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                            'ppq5Jitter', 'ddpJitter', 'localShimmer', 
                                            'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                            'apq11Shimmer', 'ddaShimmer', 'f1_mean', 'f2_mean', 
                                            'f3_mean', 'f4_mean', 'f1_median', 
                                            'f2_median', 'f3_median', 'f4_median','chroma_stft','spectral_centroid',
                                            'spectral_bandwidth','spectral_rolloff','zero_crossing_rate','chroma_cqt',
                                            'chroma_cens','melspectrogram','mfcc','spectral_contrast',
                                            'poly_features','tonnezt','AvgPower','spectral_flatness',])
    return features_Pataka

In [1]:
#Neurodegenarative Pataka test
#Pataka_dir = r"Dataset/Neurodegenerative_Diseases/test5_PaTaKa"
#Pataka_dir = r"test_5_pataka_healthy"
Pataka_audio = glob.glob(Pataka_dir + '/*.wav')
Pataka_for_ALS_PD_Combined = extract_feature(Pataka_audio)
Pataka_for_ALS_PD_Combined_without_NAN = Pataka_for_ALS_PD_Combined.dropna()
#Pataka_for_ALS_PD_Combined_without_NAN.to_csv('Dataset/healthy_data/test_5_pataka_healthy/features_for_healthy.csv')
Pataka_for_ALS_PD_Combined_without_NAN.to_csv('features_for_ND_T5_Pataka.csv')